In [1]:
## Import Modules 

import torch
import torchvision
from torch import nn, optim
import torch.nn.functional as F
from torchsummary import summary

In [2]:
## Model Configuration (Initialize hyperparameters)
batch_size = 64
learning_rate = 0.01
cross_entropy = nn.CrossEntropyLoss()

In [3]:
## DataLoader (Load the training set and validation set using Dataset and DataLoader)
transform = torchvision.transforms.ToTensor()
train_data = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(
    'mnist_data', train=True, download=True, transform=transform
    ), batch_size=batch_size
)
val_data = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(
    'mnist_data', train=False, download=True, transform=transform
    ), batch_size=batch_size
)

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
# Validation function (To check whether the model is learning properly we can use a validation set)
def validate(model, data):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(data):
        images = images.cuda()
        labels = labels.cuda()
        y_pred = model(images)
        value, pred = torch.max(y_pred, 1)
        total += y_pred.size(0)
        correct += torch.sum(pred == labels)
    return correct * 100 / total
    

In [5]:
## Training Function (Training the model)
def train(model,epochs=5) :
    optimizer = optim.Adam(model.parameters(),lr=learning_rate)    
    for n in range(epochs)  :
        for i , (images , labels) in enumerate(train_data) :
            images = images.cuda()
            labels = labels.cuda()
            optimizer.zero_grad()
            prediction = model(images)
            loss = cross_entropy(prediction, labels)
            loss.backward()
            optimizer.step()
        accuracy = float(validate(model, val_data))
        print("Epoch:", n+1, "Loss: ", float(loss.data), "Accuracy:", accuracy)

In [6]:
## Model (Define Model)
class ANN(nn.Module) :
    def __init__(self):
        super(ANN,self).__init__()
        self.dense_1 = nn.Linear(in_features=784,out_features=256)
        self.dense_2 = nn.Linear(in_features=256,out_features=10)

        self.relu = nn.ReLU()
    def forward(self,x) :
        x = x.view(x.shape[0],-1)
        x = self.relu(self.dense_1(x))
        x = self.dense_2(x)
        # output = self.tanh(x)
        output = F.log_softmax(x, dim=1)

        return output

In [7]:
# Model (Model instance)
model = ANN().cuda()

In [8]:
# Summary
summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 256]         200,960
              ReLU-2                  [-1, 256]               0
            Linear-3                   [-1, 10]           2,570
Total params: 203,530
Trainable params: 203,530
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.78
Estimated Total Size (MB): 0.78
----------------------------------------------------------------


In [9]:
# Train for 30 Epochs
train(model,epochs=30)

Epoch: 1 Loss:  0.018450777977705002 Accuracy: 94.55999755859375
Epoch: 2 Loss:  0.04523979872465134 Accuracy: 96.05999755859375
Epoch: 3 Loss:  0.004567528143525124 Accuracy: 95.05999755859375
Epoch: 4 Loss:  0.0027272189036011696 Accuracy: 95.82999420166016
Epoch: 5 Loss:  6.891608791192994e-05 Accuracy: 96.30999755859375
Epoch: 6 Loss:  0.03125550597906113 Accuracy: 92.80999755859375
Epoch: 7 Loss:  0.00023395652533508837 Accuracy: 95.95999908447266
Epoch: 8 Loss:  0.00023131839407142252 Accuracy: 96.44999694824219
Epoch: 9 Loss:  9.077379218069836e-06 Accuracy: 96.22999572753906
Epoch: 10 Loss:  0.006186465732753277 Accuracy: 96.02999877929688
Epoch: 11 Loss:  0.0006802300340496004 Accuracy: 96.94999694824219
Epoch: 12 Loss:  4.100344449398108e-05 Accuracy: 96.83999633789062
Epoch: 13 Loss:  2.2935677407076582e-05 Accuracy: 96.25999450683594
Epoch: 14 Loss:  3.574645234039053e-05 Accuracy: 96.86000061035156
Epoch: 15 Loss:  0.0023283963091671467 Accuracy: 96.5199966430664
Epoch: 16

**We can see that there are less parameters here, and the training is significantly quicker. Even after achieving zero, we can see that the loss fluctuates. Because it does not capture spatial information, the accuracy of a Normal Neural Network should be lower than that of a Convolution Neural Network. Because there are no complex characteristics to learn for the MNIST dataset, we can observe that the model achieves reasonable accuracy.**